In [158]:
import pandas as pandas
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
 

In [179]:
from 'gmcr-py'.data_01_conflictModel as model

SyntaxError: invalid syntax (<ipython-input-179-aa1160d1c124>, line 1)

In [176]:
%reload_ext autoreload
%autoreload 2
import data_01_conflictModel as model
import data_02_conflictSolvers as solvers
import data_03_gmcrUtilities as gmcrUtils
import gmcr
import numpy as np

ModuleNotFoundError: No module named 'data_01_conflictModel'

In [159]:
%reload_ext autoreload
%autoreload 2
from utilities import *

In [160]:
import os
os.getcwd()

'c:\\Users\\sahit\\Google Drive\\EPA\\Thesis\\susquehanna\\susquehanna_python\\notebooks'

### 0. Load payoff data from running the best optimization results 

In [161]:
payoffs = load_data("../model/data/processed/50yr_samples.pickle")

In [166]:
payoffs

,hydropower_revenue,atomic_power_plant_discharge,baltimore_discharge,chester_discharge,recreation,environment,flood_duration
best_app,33.428360,0.675815,0.450959,0.602542,0.750714,-0.116995,-1.666667
best_ch,44.640454,0.667632,0.581992,0.578373,0.768214,-0.045616,-1.000000
best_comp,47.521088,0.675057,0.408810,0.581773,0.839643,-0.032985,-0.833333
best_env,41.245030,0.669965,0.499875,0.603974,0.768214,-0.086025,-1.666667
best_rec,42.795952,0.675009,0.577004,0.604840,0.750714,-0.083129,-1.241667


In [165]:
payoffs = payoffs.drop('best_hydro')

In [167]:
payoffs = payoffs.reset_index(drop = True)

#### 0.1 Borrow Baseline Values from (Guiliani et. al., 2014)
Assumption: Flood Duration in baseline value is assumed to be 2 days 

In [168]:
baseline = {'hydropower_revenue':39.30, 'atomic_power_plant_discharge':0.633,
       'baltimore_discharge':1.00, 'chester_discharge':1.00, 'recreation':0.00, 'environment':0.1058,
       'flood_duration':1.633}

In [169]:
payoffs = payoffs.append(baseline, ignore_index = True)

In [170]:
normalized_baseline = normalize(payoffs).iloc[5].to_dict()
normalized_baseline

{'decision': 5.0,
 'hydropower_revenue': 0.41664325086698106,
 'atomic_power_plant_discharge': 0.0,
 'baltimore_discharge': 1.0,
 'chester_discharge': 1.0,
 'recreation': 0.0,
 'environment': 1.0,
 'flood_duration': 0.9999999999999999}

In [171]:
%reload_ext autoreload
%autoreload 2
from non_cooperative_games import *
player2 = 'hydropower_revenue'
player1 = 'atomic_power_plant_discharge'
A, B, game_outcomes = payoff_matrix(player1, player2, normalized_baseline, normalize(payoffs))
G = graphs(A, B)

equilibrium = list(dict.fromkeys([game_outcomes[n]['decision'] for n,v in G.nodes(data = True) if v['out_arrows'] == 0]))
print(f'Nash equilibrium is {equilibrium}')
gmr_p1 = gmr(G, 1,'u1')
print(f'GMR for {player1} is', [game_outcomes[x] for x in gmr_p1])
gmr_p2 = gmr(G, 2,'u2')
print(f'GMR for {player2} is', [game_outcomes[x] for x in gmr_p2])

6 decisions
Nash equilibrium is ['baseline', 1, 2, 3, 4, 5]
GMR for atomic_power_plant_discharge is [{'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}]
GMR for hydropower_revenue is [{'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision': 'baseline'}, {'decision

In [172]:
payoffs[[player1, player2]]

,atomic_power_plant_discharge,hydropower_revenue
0,0.675815,33.428360
1,0.667632,44.640454
2,0.675057,47.521088
3,0.669965,41.245030
4,0.675009,42.795952
5,0.633000,39.300000


In [175]:
normalize(payoffs)[[player1, player2]].rank().sort_values(by = "hydropower_revenue", ascending = False)

,atomic_power_plant_discharge,hydropower_revenue
2,5.0,6.0
1,2.0,5.0
4,4.0,4.0
3,3.0,3.0
5,1.0,2.0
0,6.0,1.0


In [95]:
payoffs

,index,hydropower_revenue,atomic_power_plant_discharge,baltimore_discharge,chester_discharge,recreation,environment,flood_duration
0,1.0,44.640454,0.667632,0.581992,0.578373,0.768214,-0.045616,-1.000000
1,2.0,47.521088,0.675057,0.408810,0.581773,0.839643,-0.032985,-0.833333
2,NaN,39.300000,0.633000,1.000000,1.000000,0.000000,0.105800,1.633000
